### CNN Model Training with CIFAR10 - Dataset Load & Setting

In [4]:
import torch
from torchvision import datasets as D
from torchvision import transforms as T
from torch.utils.data import random_split

transform_train = T.Compose([
     #T.RandomCrop(32, padding=4),
     #T.RandomHorizontalFlip(),
    # T.RandomRotation(15),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = T.Compose([
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

batch_size = 128 # 이미지 묶음 당 이미지 개수

#train data 정의
trainset = D.CIFAR10(root='./data', train=True,
                                        download=False, transform=transform_train) 

# trainloader를 통해 train_data를 load
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True)

#test data 정의
testset = D.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform_test)

train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
train_dataset, val_dataset = random_split(trainset, [train_size, val_size])
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

### Pretrained model - VGG16

In [5]:
from torchvision import models
import torch
import torch.nn as nn 
vgg16_model = models.vgg16(pretrained=True)
vgg16_model.classifier[6] = nn.Linear(in_features=4096, out_features=10)

### Training 과정

In [6]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import copy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = vgg16_model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=0)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

# Early stopping parameters
patience = 20
best_val_loss = float('inf')
best_model_weights = copy.deepcopy(model.state_dict())
early_stop = False

num_epochs = 200

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, targets in trainloader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_loss = running_loss / len(trainloader)

    # Validate the model
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.7f}, Val Loss: {val_loss:.7f}')

    # Check for improvement
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_weights = copy.deepcopy(model.state_dict())
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    # Early stopping
    if epochs_no_improve >= patience:
        print('Early stopping!')
        early_stop = True
        break

    # Step the scheduler
    scheduler.step(val_loss)

# Load best model weights if early stopped
if early_stop or epoch == num_epochs - 1:
    model.load_state_dict(best_model_weights)


Epoch [1/200], Train Loss: 0.7715975, Val Loss: 0.4513826
Epoch [2/200], Train Loss: 0.4316541, Val Loss: 0.3002174
Epoch [3/200], Train Loss: 0.2974399, Val Loss: 0.1746967
Epoch [4/200], Train Loss: 0.2044250, Val Loss: 0.1185046
Epoch [5/200], Train Loss: 0.1476511, Val Loss: 0.0848632
Epoch [6/200], Train Loss: 0.1132706, Val Loss: 0.0775799
Epoch [7/200], Train Loss: 0.0872473, Val Loss: 0.0417510
Epoch [8/200], Train Loss: 0.0729149, Val Loss: 0.0523715
Epoch [9/200], Train Loss: 0.0642783, Val Loss: 0.0534196
Epoch [10/200], Train Loss: 0.0511497, Val Loss: 0.0628072
Epoch [11/200], Train Loss: 0.0514308, Val Loss: 0.0298368
Epoch [12/200], Train Loss: 0.0436152, Val Loss: 0.0297743
Epoch [13/200], Train Loss: 0.0464945, Val Loss: 0.0244382
Epoch [14/200], Train Loss: 0.0394110, Val Loss: 0.0520219
Epoch [15/200], Train Loss: 0.0414222, Val Loss: 0.0396762
Epoch [16/200], Train Loss: 0.0380782, Val Loss: 0.0251278
Epoch [17/200], Train Loss: 0.0361249, Val Loss: 0.0202630
Epoch 

### CNN model Inference

In [7]:
def inference(model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        
        for inputs, labels in testloader:
            images, labels = inputs.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the test images: {100 * correct / total} %')

In [8]:
inference(model)

Accuracy of the network on the test images: 89.66 %
